In [15]:
#from __future__ import print_function

import re
from bs4 import BeautifulSoup
import urllib.request
import pprint
import datetime
import os
import pandas as pd

today = datetime.datetime.today()
today_formal = today.strftime("%A_%d_%B_%Y_%I%M_%p")
print(today_formal)
# importing necessary packages

example_table_link = "https://www.fda.gov/Drugs/DrugSafety/DrugRecalls/default.htm"


def to_string(s):
    """
    makes a string
    input example: 10000
    output example: '1000'
    :type s: what you want to be a sting
    """
    return str(s)


def remove_control_chart(s):
    """
    :param s: string that may not be utf-8 encode
    :return:
    """
    #s = s.replace(',', ' ')
    s = re.sub(r'\n', '', s)
    s = s.replace('\xa0', '')
    s = re.sub(r' +', ' ', s)
    #s = '"' + s + '"'
    return to_string(s)


def clean_up_cols(s):
    """
    :param s: string for example # AFFECTED
    :return: space removed for example number_affected
    """
    s = to_string(s)
    s = s.upper()
    s = s.replace(' ', '_')
    s = s.replace('/', '')
    s = s.replace('#', 'number')
    #s = '"' + s + '"'
    return s


def WEBSCRAPeTABLES(link, outputfile_name):
    """
    :param outputfile_name: path to where you want your file to be saved, for example 'table.csv'
    :param link: this will be the link where you want to scrape table
    :return: json and csv that returns metadata or table data with header keys
    """
    cnames = []
    with urllib.request.urlopen(link) as response:
        fout = open(outputfile_name, 'w')
        the_page = response.read()
        bs = BeautifulSoup(the_page, features="lxml")
        table = bs.find(lambda tag: tag.name == 'table')
        table_tr = table.tr
        list_columns = table_tr.find_all('th')
        # this provides the list of columns using the clean_up_cols fxn defined above
        for lc in list_columns:
            cnames.append(clean_up_cols(lc.text.strip()))
        rows = table.findAll(lambda tag: tag.name == 'tr')
        counter = 0
        ls = []
        for row_content in rows:
            counter += 1
            values = row_content.find_all('td')
            csv_row = [val.text.strip() for val in values]

            ls.append(csv_row)
    df = pd.DataFrame(ls)
    df = df.dropna()
    df.columns = cnames
    print("\n\n*******\n\nwrote out", to_string(counter), "rows from table\n in csv file called " + outputfile_name)
    df.to_csv(outputfile_name, index=False)

name = "FDA_DrugRecalls_" + to_string(today_formal) + ".csv" #+ to_string(example_table_link.split('/')[-2]) + "_"+ to_string(today_formal)  + ".csv"
print(name)
WEBSCRAPeTABLES(example_table_link, name)



Tuesday_06_November_2018_0310_AM
FDA_DrugRecalls_Tuesday_06_November_2018_0310_AM.csv


*******

wrote out 73 rows from table
 in csv file called FDA_DrugRecalls_Tuesday_06_November_2018_0310_AM.csv


In [16]:
df_fda =pd.read_csv(name)
df_fda.sample(10)

,DATE,BRAND_NAME,PRODUCT_DESCRIPTION,REASON_PROBLEM,COMPANY,RECALL_TYPE
0,11/02/2018,ORTHO-NOVUM,ORTHO-NOVUM 1/35 and 7/7/7\r\n(norethindrone /...,does not include the appropriate instructions ...,"Janssen Pharmaceuticals, Inc.","454, 208"
4,10/15/2018,Zero Xtreme,Dietary Supplement for Weight Loss,Undeclared Sibutramine,Fat Burners Zone,"454, 208"
66,12/19/2017,AuroMedics Pharma LLC,Pantoprazole Sodium for Injection 40 mg per vial,Contains glass particles,AuroMedics Pharma LLC,208
12,09/05/2018,Beaumont Bio Med Homeopathic,Homeopathic oral sprays,Microbial contamination,"Beaumont Bio Med, Inc.",208
20,08/22/2018,"KB, DK, more",Homeopathic products,Microbial contamination,King Bio,208
7,10/03/2018,"Pet Alive, Native Remedies, Healthful Naturals",Homeopathic Drug Products (liquid),Microbial Contamination,"Silver Star Brands, Inc.","208, 183, 495"
13,09/05/2018,Camber Pharmaceuticals,"Montelukast Sodium Tablets, USP 10mg",Product label mix up/may contain Losartan Pota...,"Camber Pharmaceuticals, Inc",208
55,02/15/2018,AVpak,"Acyclovir Tablet, USP, 400mg, 50ct Unit Dose",Product mix-up,Apace Packaging LLC,"454, 208"
8,09/28/2018,Endo Pharmaceuticals,"Robaxin® (methocarbamol tablets, USP) 750mg Ta...",Due to Incorrect Daily Dosing Information on L...,Endo Pharmaceuticals,208
52,03/05/2018,"Hospira, Inc.","Hydromorphone HCL Injection, USP",Units may be empty or cracked at the bottom of...,"Hospira, Inc.",208
